In [1]:
# importing libraries
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib as mpl
import warnings
warnings.filterwarnings('ignore')

In [3]:
# reading all required csv files
# for reading a csv file that we get from scrapping we folloe path Data/Play-by-play data/team_name/series_game_number.csv

# reading data of series played betweeen buks and bulls
df_game_1 = pd.read_csv('Data/Play-by-play data/Buks_vs_bulls/buks_vs_bulls_series_game1.csv')
# df_game_2 = pd.read_csv('Data/Play-by-play data/Buks_vs_bulls/buks_vs_bulls_series_game2.csv')
# df_game_3 = pd.read_csv('Data/Play-by-play data/Buks_vs_bullsbuks_vs_bulls_series_game3.csv')
# df_game_4 = pd.read_csv('Data/Play-by-play data/Buks_vs_bullsbuks_vs_bulls_series_game4.csv')
# df_game_5 = pd.read_csv('Data/Play-by-play data/Buks_vs_bullsbuks_vs_bulls_series_game5.csv')

# reading data of series played between suns and orleans
# df_game_1 = pd.read_csv('Data/Play-by-play data/suns_vs_orleans/Suns_vs_peli_series_Game1.csv')
# df_game_2 = pd.read_csv('Data/Play-by-play data/suns_vs_orleans/Suns_vs_peli_series_Game2.csv')
# df_game_3 = pd.read_csv('Data/Play-by-play data/suns_vs_orleans/Suns_vs_peli_series_Game3.csv')
# df_game_4 = pd.read_csv('Data/Play-by-play data/suns_vs_orleans/Suns_vs_peli_series_Game4.csv')
# df_game_5 = pd.read_csv('Data/Play-by-play data/suns_vs_orleans/Suns_vs_peli_series_Game5.csv')
# df_game_6 = pd.read_csv('Data/Play-by-play data/suns_vs_orleans/Suns_vs_peli_series_Game6.csv')
# df_game_7 = pd.read_csv('Data/Play-by-play data/suns_vs_orleans/Suns_vs_peli_series_Game7.csv')
df_game_1.head()

,Unnamed: 0,Time,Chicago,Unnamed: 3,Score,Unnamed: 5,Milwaukee
0,2,12:00.0,Jump ball: N. Vucevic vs. B. Lopez (W. Matthew...,Jump ball: N. Vucevic vs. B. Lopez (W. Matthew...,Jump ball: N. Vucevic vs. B. Lopez (W. Matthew...,Jump ball: N. Vucevic vs. B. Lopez (W. Matthew...,Jump ball: N. Vucevic vs. B. Lopez (W. Matthew...
1,3,11:41.0,NaN,NaN,0-0,NaN,Shooting foul by A. Caruso (drawn by B. Lopez)
2,4,11:41.0,NaN,NaN,0-1,+1,B. Lopez makes free throw 1 of 2
3,5,11:41.0,NaN,NaN,0-2,+1,B. Lopez makes free throw 2 of 2
4,6,11:24.0,Z. LaVine misses 2-pt layup from 1 ft,NaN,0-2,NaN,NaN


In [80]:
# taking one dataframe at a time 
df_game = df_game_5.copy() # create a copy of dataframe

# deleting unnecessary columns
del df_game['Unnamed: 0']
del df_game['Unnamed: 3']
del df_game['Unnamed: 5']

df_game.fillna('NA', inplace=True) # replacing null values by 'NA'
df_game = df_game.iloc[1: , :] # deleting the first row of dataframe, because it has some text data that we not need

df_game['quarter'] = np.nan # creating an empty column for quarter
# df_game['Series Game'] = 1
df_game['Series Game'] = 5 # column for series games

In [82]:
# by using the text "End of 1st quarter", we extract the number 1 and assign it to the respected rows
# here we select "Milwaukee" column, it changes according to the player and team
df_game.loc[df_game['Milwaukee'] == 'End of 1st quarter', 'quarter'] = 1
df_game.loc[df_game['Milwaukee'] == 'End of 2nd quarter', 'quarter'] = 2
df_game.loc[df_game['Milwaukee'] == 'End of 3rd quarter', 'quarter'] = 3
df_game.loc[df_game['Milwaukee'] == 'End of 4th quarter', 'quarter'] = 4

df_game.loc[0:df_game[df_game['quarter'] == 1].index[0]]['quarter'].fillna(1, inplace = True)
df_game.loc[df_game[df_game['quarter'] == 1].index[-1]:df_game[df_game['quarter'] == 2].index[0]]['quarter'].fillna(2, inplace = True)
df_game.loc[df_game[df_game['quarter'] == 2].index[-1]:df_game[df_game['quarter'] == 3].index[0]]['quarter'].fillna(3, inplace = True)
df_game['quarter'].fillna(4, inplace = True)

In [87]:
# Dataframe for makes
# here we select "Milwaukee" column, it changes according to the player and team
data_makes = df_game[df_game['Milwaukee'].str.contains('G. Antetokounmpo makes')][['Time', 'Milwaukee', 'Score', 'quarter', 'Series Game']]
data_makes.reset_index(inplace = True, drop = True)
# data_makes.head()

# creating and concating data for makes
df_col_makes = pd.DataFrame(data_makes['Milwaukee'].str.split().tolist(), columns = ['Name1', 'Name2', 'misses/count', 'basket_type',
                                                               'short_type','Misc1', 'Misc2', 'short_range', 'Misc3', 'Assist',
                                                                  'Misc4', 'Misc5', 'Assist_by'])

concat_df_makes = pd.concat([data_makes, df_col_makes], axis = 1)
concat_df_makes['player_name'] = concat_df_makes['Name1'] + concat_df_makes['Name2'] #player name
concat_df_makes['player_team'] = 'Milwaukee' # team name 

concat_df_makes.loc[concat_df_makes['short_range'] == 'ft', 'short_range'] = concat_df_makes['Misc2']
# Selcting only usefull columns
makes_final_df = concat_df_makes[['Time', 'player_name', 'player_team', 'Series Game', 'Score', 'quarter', 
                                  'basket_type', 'short_type', 'short_range','misses/count']]

In [90]:
# Dataframe for misses
data_misses = df_game[df_game['Milwaukee'].str.contains('G. Antetokounmpo misses')][['Time', 'Milwaukee', 'Score', 'quarter', 'Series Game']]
data_misses.reset_index(inplace = True, drop = True)
# data_misses.head()

# creating and concating data for missies
df_misses = pd.DataFrame(data_misses['Milwaukee'].str.split().tolist(), columns = ['Name1', 'Name2', 'misses/count',
                        'basket_type', 'short_type','Misc1', 'Misc2', 'short_range', 'ft',
                            'Misc3', 'Misc4', 'Def_name_1'])#, 'Def_name_2'])

df_misses['player_name'] = df_misses['Name1'] + df_misses['Name2']
concat_df = pd.concat([data_misses, df_misses], axis = 1)
# selcting usefull columns
misses_final_df = concat_df[['Time', 'player_name','Series Game', 'Score','quarter', 'basket_type', 'short_type', 'short_range',
                              'misses/count']]
misses_final_df['player_team'] = 'Milwaukee'
makes_misses_df = misses_final_df.append(makes_final_df)
makes_misses_df.reset_index(drop = True, inplace = True)

In [92]:
# creating a dummy time stamp variable for arranging the shorts in a propar way
makes_misses_df['Time_dumy'] = pd.to_datetime(makes_misses_df['Time'])

In [93]:
# creating a new Data Frame
arrange_df = pd.DataFrame()

# appending all data by quater wise and sort the values according to time variabe, so that all the shorts are in sequence.
arrange_df = arrange_df.append(makes_misses_df[makes_misses_df['quarter'] == 1].sort_values(by = 'Time_dumy', ascending = False))
arrange_df = arrange_df.append(makes_misses_df[makes_misses_df['quarter'] == 2].sort_values(by = 'Time_dumy', ascending = False))
arrange_df = arrange_df.append(makes_misses_df[makes_misses_df['quarter'] == 3].sort_values(by = 'Time_dumy', ascending = False))
arrange_df = arrange_df.append(makes_misses_df[makes_misses_df['quarter'] == 4].sort_values(by = 'Time_dumy', ascending = False))

In [23]:
# creating a new dataframe (but only for one time, that just append all games data into it, after that we save it in a csv file.)
all_games_df = pd.DataFrame()

In [ ]:
# re run this append command for every series game, so that we can have all data into one dataframe
all_games_df = all_games_df.append(arrange_df)

In [98]:
# reset the index and save dataframe into a csv file
all_games_df.reset_index(drop = True, inplace = True)
all_games_df.to_csv('Data/preprocessed_data/giannis_datagiannis_series_data(buks_vs_bulls)_data.csv', index = False)

In [ ]:
# for saving csv files we follow path Data/preprocessed_data/"player_name"_data
# example Data/preprocessed_data/giannis_data # for giannis data